### Create Sample DataFrame

In [0]:
data_student = [
    ("Alice", "Physics", 85, "Pass", 90),
    ("Alice", "Chemistry", 78, "Pass", 88),
    ("Alice", "Maths", 92, "Pass", 95),
    ("Bob", "Physics", 80, "Pass", 85),
    ("Bob", "Chemistry", 75, "Fail", 80),
    ("Bob", "Maths", 88, "Pass", 92),
    ("Charlie", "Physics", 90, "Pass", 93),
    ("Charlie", "Chemistry", 82, "Pass", 89),
    ("Charlie", "Maths", 87, "Pass", 91),
    ("Diana", "Physics", 70, "Fail", 78),
    ("Diana", "Chemistry", 85, "Pass", 90),
    ("Diana", "Maths", 80, "Pass", 88),
    ("Ethan", "Physics", 88, "Pass", 92),
    ("Ethan", "Chemistry", 79, "Pass", 85),
    ("Ethan", "Maths", 91, "Pass", 94)
]

schema = ["Name", "Subject", "Marks", "Status", "Attendance"]
df_student = spark.createDataFrame(data_student, schema)

display(df_student)

Name,Subject,Marks,Status,Attendance
Alice,Physics,85,Pass,90
Alice,Chemistry,78,Pass,88
Alice,Maths,92,Pass,95
Bob,Physics,80,Pass,85
Bob,Chemistry,75,Fail,80
Bob,Maths,88,Pass,92
Charlie,Physics,90,Pass,93
Charlie,Chemistry,82,Pass,89
Charlie,Maths,87,Pass,91
Diana,Physics,70,Fail,78


### Create Rank Within each Group Of NAme

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

windowDept = Window.partitionBy("Name").orderBy(df_student.Marks.desc())
df2 = df_student.withColumn("Row", row_number().over(windowDept)).orderBy("Name", "Row")
display(df2)

Name,Subject,Marks,Status,Attendance,Row
Alice,Maths,92,Pass,95,1
Alice,Physics,85,Pass,90,2
Alice,Chemistry,78,Pass,88,3
Bob,Maths,88,Pass,92,1
Bob,Physics,80,Pass,85,2
Bob,Chemistry,75,Fail,80,3
Charlie,Physics,90,Pass,93,1
Charlie,Maths,87,Pass,91,2
Charlie,Chemistry,82,Pass,89,3
Diana,Chemistry,85,Pass,90,1


### Get Top N Rows per group of name

In [0]:
df3 = df2.filter(col("Row")==1) # <=1, <=2
display(df3)

Name,Subject,Marks,Status,Attendance,Row
Alice,Maths,92,Pass,95,1
Bob,Maths,88,Pass,92,1
Charlie,Physics,90,Pass,93,1
Diana,Chemistry,85,Pass,90,1
Ethan,Maths,91,Pass,94,1


### Create Rank Within each Group Of Subject

In [0]:
windowDept = Window.partitionBy("Subject").orderBy(df_student.Marks.desc())
df4 = df_student.withColumn("Row", row_number().over(windowDept))
display(df4)

Name,Subject,Marks,Status,Attendance,Row
Diana,Chemistry,85,Pass,90,1
Charlie,Chemistry,82,Pass,89,2
Ethan,Chemistry,79,Pass,85,3
Alice,Chemistry,78,Pass,88,4
Bob,Chemistry,75,Fail,80,5
Alice,Maths,92,Pass,95,1
Ethan,Maths,91,Pass,94,2
Bob,Maths,88,Pass,92,3
Charlie,Maths,87,Pass,91,4
Diana,Maths,80,Pass,88,5


%md
### Get Top N Rows per group of name

In [0]:
df5 = df4.filter(col("Row")==1) # <=1, <=2
df5.display()

Name,Subject,Marks,Status,Attendance,Row
Alice,Maths,92,Pass,95,1
Charlie,Physics,90,Pass,93,1
Diana,Chemistry,85,Pass,90,1


### Reverse Rank to Get Bottom N Rows Per Group

In [0]:
windowDept = Window.partitionBy("Subject").orderBy(df_student.Marks)
df6 = df_student.withColumn("Row", row_number().over(windowDept))
display(df6)

Name,Subject,Marks,Status,Attendance,Row
Bob,Chemistry,75,Fail,80,1
Alice,Chemistry,78,Pass,88,2
Ethan,Chemistry,79,Pass,85,3
Charlie,Chemistry,82,Pass,89,4
Diana,Chemistry,85,Pass,90,5
Diana,Maths,80,Pass,88,1
Charlie,Maths,87,Pass,91,2
Bob,Maths,88,Pass,92,3
Ethan,Maths,91,Pass,94,4
Alice,Maths,92,Pass,95,5


### Get bottom N Rows per group

In [0]:
df7 = df6.filter(col("Row")==1) # <=1, <=2
df7.display()

Name,Subject,Marks,Status,Attendance,Row
Bob,Chemistry,75,Fail,80,1
Diana,Maths,80,Pass,88,1
Diana,Physics,70,Fail,78,1
